In [1]:
# Conecta ao drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Importa bibliotecas necessárias
import json
import pandas as pd
import re
import html

# Carrega o dataset
data = []
with open('/content/drive/MyDrive/FIAP/TC3/trn.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# Seleciona apenas as colunas "title" e "content"
df = df[['title', 'content']]

# Remove linhas com "content" vazio
df = df[df['content'].notna() & (df['content'] != '')]

def limpar_texto(text):
    # Decodifica entidades HTML
    text = html.unescape(text)

    # Remove tags HTML restantes
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Substitui hífens duplos por um espaço
    text = text.replace('--', ' ')

    # Remove espaços em branco extras
    text = ' '.join(text.split())

    return text

df['content'] = df['content'].apply(limpar_texto)
df['title'] = df['title'].apply(limpar_texto)

# Remove duplicatas
df = df.drop_duplicates()

# Remove linhas muito grandes
df = df[df['content'].str.len() < 1024]

# Reseta o índice
df = df.reset_index(drop=True)

# Prepara os prompts para o fine-tuning
def format_prompt(row):
    if not row['title'] or not row['content']:
        return None
    return f"<s>[INST] Gere uma descrição para o seguinte produto: {row['title']} [/INST] {row['content']} </s>"

df['prompt'] = df.apply(format_prompt, axis=1)

# Remove linhas onde o prompt não pôde ser criado
df = df.dropna(subset=['prompt'])

# Limita o dataframe a 200.000 linhas
df_limited = df.head(200000)

# Exibe um exemplo do resultado limpo e formatado
print(df_limited.head()['prompt'].iloc[0])
print("-" * 80)

# Salva o dataset preparado para uso futuro
df_limited.to_csv('/content/drive/MyDrive/FIAP/TC3/trn.csv', index=False)

print(f"\nDataset preparado com sucesso! Total de amostras: {len(df_limited)}")

<s>[INST] Gere uma descrição para o seguinte produto: Girls Ballet Tutu Neon Pink [/INST] High quality 3 layer ballet tutu. 12 inches in length </s>
--------------------------------------------------------------------------------

Dataset preparado com sucesso! Total de amostras: 200000


In [10]:
# Instala bibliotecas necessárias
!pip install -q transformers datasets peft bitsandbytes trl accelerate

In [12]:
# Importa bibliotecas necessárias
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configuração para carregar o modelo em 4-bit
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto" # Mapeia o modelo para a GPU automaticamente
)

# Teste de baseline
prompt_teste = "Gere uma descrição para o seguinte produto: Girls Ballet Tutu Neon Pink"
input_ids = tokenizer.encode(f"<s>[INST] {prompt_teste} [/INST]", return_tensors="pt").to("cuda")

output = model.generate(input_ids, max_length=100, num_return_sequences=1)
resposta_baseline = tokenizer.decode(output[0], skip_special_tokens=True)

print("Resposta do Modelo ANTES do Fine-Tuning")
print(resposta_baseline)

Resposta do Modelo ANTES do Fine-Tuning
[INST] Gere uma descrição para o seguinte produto: Girls Ballet Tutu Neon Pink [/INST] Gere uma descrição para o seguinte produto: Girls Ballet Tutu Neon Pink [/INST] Gere uma descrição para o seguinte produto: Girls Ballet Tutu Neon Pink [/INST] Gere uma descrição para o seguinte produto: Girls Ballet Tut


In [14]:
# Importa bibliotecas necessárias
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments

# Carrega o dataset
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/FIAP/TC3/trn.csv', split='train')

# Renomeia a coluna 'prompt' para 'text'
dataset = dataset.rename_column("prompt", "text")

# Configuração do LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Adiciona adaptadores LoRA ao modelo
model = get_peft_model(model, lora_config)

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FIAP/TC3/resultados_fine_tuning",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    save_total_limit=2,
    overwrite_output_dir=True,
)

# Configura o tokenizer para fine-tuning
tokenizer.pad_token = tokenizer.eos_token

# Cria o treinador SFT
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Inicia o treinamento
print("Iniciando o treinamento...")
trainer.train()
print("Treinamento concluído com sucesso!")

# Salva o modelo treinado
trainer.save_model("/content/drive/MyDrive/FIAP/TC3/modelo_final")
print("Modelo salvo com sucesso!")

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Iniciando o treinamento...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rafaguido (rafaguido-fiap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.730800
20,2.228600
30,2.222800
40,2.101000
50,2.054800
60,2.069000
70,2.109600
80,2.118900
90,2.101600
100,2.048200


Treinamento concluído com sucesso!
Modelo salvo com sucesso!


In [41]:
# Importa biblioteca necessária
from peft import PeftModel

# Carrega o modelo base
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Carrega os adaptadores LoRA treinados
peft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/FIAP/TC3/modelo_final")

# Função para gerar respostas
def gerar_descricao(titulo_produto):
    prompt = f"<s>[INST] Gere uma descrição para o seguinte produto: {titulo_produto} [/INST]"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    # Gera a resposta
    output = peft_model.generate(
        input_ids=input_ids,
        max_length=150,
        temperature=0.7, # Controla a criatividade
        top_p=0.9,
        do_sample=True
    )
    resposta = tokenizer.decode(output[0], skip_special_tokens=True)
    # Limpa o prompt da resposta final
    resposta_limpa = resposta.split("[/INST]")[-1].strip()
    return resposta_limpa

# Teste final
novo_produto = "Winnie the Pooh Pooh's Easter Basket"
descricao_gerada = gerar_descricao(novo_produto)

print(f"\nDescrição gerada para: {novo_produto}")
print(descricao_gerada)


Descrição gerada para: Winnie the Pooh Pooh's Easter Basket
The Pooh's Easter basket is the perfect gift for any Pooh fan. It features the beloved characters, including Pooh, Piglet, Eeyore, and Tigger. Included is a chocolate Easter egg with a miniature Pooh and Piglet inside. The basket also has a removable tray for holding smaller items such as Easter eggs or chocolates. This basket is perfect for gift giving and is sure to bring joy to the recipient.
